In [4]:
''' IMPORT ALL THE THINGS '''
import os
import matplotlib.pyplot as plt
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img, img_to_array, smart_resize
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import layers
from tensorflow.keras.applications.vgg19 import VGG19

In [5]:
''' DEFINE VARIABLES '''

CWD = os.getcwd().replace('\\','/') # get current workspace directory
IMAGE_DIR = f'{CWD}/images'
TRAIN_DATASET_DIR = f'{IMAGE_DIR}/train'
TEST_DATASET_DIR = f'{IMAGE_DIR}/test'
VAL_DATASET_DIR = f'{IMAGE_DIR}/validation'

CATEGORIES = ['clementine','grapefruit','orange']

IMAGE_RESIZE = (224, 224) # size of image to put in Model

AUGMENT_N_IMAGE = 4 # how many time each image get the augmentation



In [6]:
''' Create Data Augmentation '''
def run_augmentation( isRun = 0 ):

    if isRun:

        print('Running Augmentation...\n')

        datagen = ImageDataGenerator(
            rotation_range=45,
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            fill_mode='reflect' # optional : nearest, constant(125), reflect, wrap
        ) 

        ''' METHOD 1 DOES NOT WORK , don't know why '''
        # aug_train = datagen.flow_from_directory(
        #     TRAIN_DATASET_DIR,
        #     target_size=IMAGE_RESIZE,
        #     class_mode='categorical',
        #     batch_size=1,
        #     save_to_dir=f"{TRAIN_DATASET_DIR}/preview",
        #     save_prefix='aug',
        #     save_format='jpeg'
        # )
        ''' METHOD 2 '''
        try:
            for category in CATEGORIES:

                train_image_in_directory = [] 

                train_category_directory = f"{TRAIN_DATASET_DIR}/{category}"

                for i, image_name in enumerate(os.listdir(train_category_directory)):
                    
                    file_path = f"{train_category_directory}/{image_name}"
                    if (image_name.find('aug') != -1): # delete old augmentation
                        if (os.path.exists(file_path)):
                            os.remove(file_path)

                    elif (image_name.split('.')[1] in ['png', 'jpg', 'jpeg']):

                        # image = cv2.imread(
                        #     f'{class_directory}/{image_name}', cv2.IMREAD_UNCHANGED)       ### NOT USING OpenCV cause the image is in BGR ordering not RGB that make img BLUE
                        # image = cv2.resize(image, IMAGE_RESIZE, interpolation=cv2.INTER_AREA)

                        image = load_img(file_path)
                        image = smart_resize(image, IMAGE_RESIZE)
                        image = img_to_array(image)
                        train_image_in_directory.append(image)

                train_image_in_directory = np.array(train_image_in_directory)

                print(train_image_in_directory.shape)

                i = 0
                
                # save_to_dir (option) : f'{train_category_directory}', 'augmented'
                for batch in datagen.flow(train_image_in_directory, batch_size=1, save_to_dir=f'{train_category_directory}', save_prefix='aug', save_format='jpg'):
                    i += 1
                    if i >= AUGMENT_N_IMAGE:  # datagen.flow make infinite loop NEED A MANUAL BREAK!!!
                        break

            print('\nAugmentation Success!!!')

        except Exception as e:

            print('\nAugmentation Failed...')
            print('Exception is\n',e)
    
    


In [7]:
run_augmentation(1)  

Running Augmentation...

(1, 224, 224, 3)
(1, 224, 224, 3)
(1, 224, 224, 3)

Augmentation Success!!!


In [8]:
''' LOAD DATA into Variables '''
try:
    print("Loading data...\n")
    
    print('-----------Train-----------')
    #Load train data

    train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
        TRAIN_DATASET_DIR, # train_data path
        image_size=IMAGE_RESIZE, # each image resize
        batch_size=1,
        label_mode='categorical',
        color_mode='rgb',
        shuffle=True
    )

    print(train_dataset.class_names)  # class check
    print(train_dataset)


    print("-----------Validation-----------")
    #Load validation data
    valid_dataset = tf.keras.preprocessing.image_dataset_from_directory(
        VAL_DATASET_DIR,  # validation_data path
        image_size=IMAGE_RESIZE,  # each image resize
        batch_size=1, 
        label_mode='categorical',
        color_mode='rgb',
        shuffle=True
    )

    print(valid_dataset.class_names)  # class check
    print(valid_dataset)

    print("\nLoad data Success!!!")
    
except Exception as e:

    print("\nLoad data failed...")
    print("Exception is",e)




Loading data...

-----------Train-----------
Found 15 files belonging to 3 classes.
['clementine', 'grapefruit', 'orange']
<BatchDataset shapes: ((None, 224, 224, 3), (None, 3)), types: (tf.float32, tf.float32)>
-----------Validation-----------
Found 3 files belonging to 3 classes.
['clementine', 'grapefruit', 'orange']
<BatchDataset shapes: ((None, 224, 224, 3), (None, 3)), types: (tf.float32, tf.float32)>

Load data Success!!!


In [9]:
''' Get pre train model '''
base_model = VGG19()

In [10]:
''' Create a model '''

# new last layer
prediction_layer = layers.Dense(3, activation='softmax', name="prediction")(base_model.get_layer(index=-1).output)

# create a model from pre train model with new last layer
model = Model(inputs=base_model.input, outputs=prediction_layer)


In [11]:
# model.summary() # show all the model's layer

In [12]:
''' Compiling '''

opt = tf.keras.optimizers.Adam() ### can be changed
los = tf.keras.losses.CategoricalCrossentropy()  ### can be changed
met = tf.keras.metrics.Accuracy()  ### can be changed

model.compile(optimizer=opt, loss=los, metrics=met)


In [13]:
''' Training '''
history = model.fit(
    train_dataset,  # both data and label cause we use preprocessing
    validation_data=valid_dataset,
    epochs=2, ### can be change
    shuffle=True 
)


Epoch 1/2
15/15 [==============================] - 28s 2s/step - loss: 1.1051 - accuracy: 0.0000e+00 - val_loss: 1.1006 - val_accuracy: 0.0000e+00
Epoch 2/2


NotFoundError:  NewRandomAccessFile failed to Create/Open: d:/Workspace/OrangeClassificationDeepLearning/src/images/train\orange\aug_0_7478.jpg : The system cannot find the file specified.
; No such file or directory
	 [[{{node ReadFile}}]]
	 [[IteratorGetNext]] [Op:__inference_train_function_2302]

Function call stack:
train_function


In [ ]:
'''Plot into graph'''
history.history

{'loss': [1.1053296327590942, 1.100548267364502],
 'accuracy': [0.0, 0.0],
 'val_loss': [1.100164771080017, 1.100172519683838],
 'val_accuracy': [0.0, 0.0]}

In [ ]:
''' Evaluating '''

' Evaluating '

In [ ]:
''' Predicting '''

' Predicting '